In [1]:
'''Get temp data to test dfs'''

import yfinance as yf
import pandas as pd


def temp_ticks(ticker,start=''):
    df = yf.download(ticker,period='5y',)
    return df

data = yf.download('GOOG',period='5y',) #pd.DataFrame

tickers_list = ['AAPL', 'AMZN', 'MSFT', 'WMT']
for ticker in tickers_list:
    sym = ticker
    ticker = yf.download(sym, period='5y',)
    ticker['symbol'] = str(sym)
    ticker.set_index('symbol')
    data.append(ticker)
    print(data)

    

    #data.merge(ticker,left_index='symbol')
    #data = pd.concat([data])
    
ticker
#data  # -- append now working.

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2015-02-11   533.834351   536.975708   531.917603   534.502502   534.502502   
2015-02-12   535.778992   543.328308   533.209045   541.443481   541.443481   
2015-02-13   541.862305   548.404358   541.642944   547.506836   547.506836   
2015-02-17   545.332764   548.494080   539.608521   541.353699   541.353699   
2015-02-18   539.917664   543.996460   536.038330   538.222290   538.222290   
...                 ...          ...          ...          ...          ...   
2020-02-05  1462.420044  1463.839966  1430.560059  1448.229980  1448.229980   
2020-02-06  1450.329956  1482.000000  1449.569946  1476.229980  1476.229980   
2020-02-07  1467.300049  1485.839966  1466.349976  1479.229980  1479.229

/Users/zoakes/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


[*********************100%***********************]  1 of 1 completed
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2015-02-11   533.834351   536.975708   531.917603   534.502502   534.502502   
2015-02-12   535.778992   543.328308   533.209045   541.443481   541.443481   
2015-02-13   541.862305   548.404358   541.642944   547.506836   547.506836   
2015-02-17   545.332764   548.494080   539.608521   541.353699   541.353699   
2015-02-18   539.917664   543.996460   536.038330   538.222290   538.222290   
...                 ...          ...          ...          ...          ...   
2020-02-05  1462.420044  1463.839966  1430.560059  1448.229980  1448.229980   
2020-02-06  1450.329956  1482.000000  1449.569946  1476.229980  1476.229980   
2020-02-07  1467.300049  1485.839966  1466.349976  1479.229980  1479.229980   
2020-02-10  1474.319946  1509.500000  1474.319946  1508.680054

,Open,High,Low,Close,Adj Close,Volume,symbol
Date,,,,,,,
2015-02-11,86.629997,87.120003,85.919998,86.339996,76.188599,5853100,WMT
2015-02-12,86.559998,86.680000,85.230003,85.889999,75.791504,6941600,WMT
2015-02-13,85.839996,86.160004,85.320000,85.809998,75.720894,5597000,WMT
2015-02-17,85.430000,85.970001,84.970001,85.959999,75.853256,5087200,WMT
2015-02-18,85.959999,86.300003,85.519997,86.290001,76.144447,6032700,WMT
...,...,...,...,...,...,...,...
2020-02-05,115.599998,116.949997,114.849998,116.809998,116.809998,5682900,WMT
2020-02-06,116.830002,117.230003,116.059998,116.309998,116.309998,5352900,WMT
2020-02-07,116.110001,116.760002,115.760002,116.449997,116.449997,3689600,WMT


In [61]:
'''Individual Spread tests (single instrument at a time)'''
import datetime
from ib_insync import *
import pandas as pd
import numpy as np

df = ticker

def get_symbol_ticks(ticker,start = ''):
    end = datetime.datetime.now()
    ticks = ib.reqHistoricalTicks(ticker,start,end,1000,'BID_ASK',useRth=True)
    df = util.df(ticks)
    return df


def get_z_spread(df,stdevs):
    if ('ask','bid') not in df.columns:
        try:
            df = df.rename(columns = {'High':'ask','Low':'bid'})
            #df = df.rename(columns = {'Ask':'ask','Bid':'bid'})
        except Exception:
            print('Error -- check column names')
    df['spread'] = df['ask'] - df['bid']
    df['spread_20'] = df['spread'].rolling(window=20).mean()
    df['sigma_20'] = df['spread'].rolling(window=20).std()
    df['sprd_up'] = df['spread_20'] + (df['sigma_20'] * stdevs)
    df['sprd_dn'] = df['spread_20'] - (df['sigma_20'] * stdevs)
    df['zscore'] = (df['spread'] - df.spread_20) / df.sigma_20
    df['z_spread'] = np.where((df.zscore > stdevs),'Hi',np.where((df.zscore < -stdevs),'Lo','Avg'))

    ZSpread = df['z_spread'].iloc[-1]
    ZScore = df['zscore'].iloc[-1]
    return ZSpread,  ZScore

get_z_spread(df,1)

#print(df)
#ZSP = {}
#for ticker in tickers:
    #df.loc[ticker,'zscore'] = df['z_spread'] = np.where((df.zscore > 1),'Hi',np.where((df.zscore < -1),'Lo','Avg'))

('Avg', -0.9586196650281728)

In [42]:
'''Run ZSpread ZScores on MULTIPLE instruments.'''

def get_symbol_tickers(tickers,start=''):
    '''Confirm append works ! (may need concat)'''
    DF = pd.DataFrame()
    end = datetime.datetime.now()
    for tick in tickers:
        temp = ib.reqHistoricalTicks(tick,start,end,1000,'BID_ASK',useRth=False)
        df = util.df(temp)
        df['symbol'] = tick
        DF.append(df)
        print(f'{tick} added to DF')
    return DF


def get_z_spread_multi(tickers,stdevs):
    #df.set_index(['symbol','date']) #Maybe groupby() cleaner?
    ZSpreads = {}
    ZScores = {}
    for tick in tickers:
        #df = temp_ticks(tick) -- to test w.out IB.
        df = get_symbol_ticks(tick) 
        zsp, zsc = get_z_spread(df,stdevs)
        ZSpreads[tick] = zsp
        ZScores[tick] = zsc
    return ZSpreads, ZScores

#get_symbol_tickers(['aapl','amzn'])

get_z_spread_multi(['AAPL','AMZN'],1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


({'AAPL': 'Hi', 'AMZN': 'Avg'},
 {'AAPL': 1.0010297286928287, 'AMZN': -0.8369496409709574})

In [2]:
'''Added ATR quickly -- TAlib may be cleaner'''

def get_z_spread_slip(df,stdevs=1.5,denom=1.5):
    if ('ask','bid') not in df.columns:
        try:
            df = df.rename(columns = {'High':'ask','Low':'bid'})
            #df = df.rename(columns = {'Ask':'ask','Bid':'bid'}) #Change col names to correct case if needed
        except Exception:
            print('Error -- check column names')
    df['spread'] = df['ask'] - df['bid']
    df['spread_20'] = df['spread'].rolling(window=20).mean()
    df['sigma_20'] = df['spread'].rolling(window=20).std()
    df['sprd_up'] = df['spread_20'] + (df['sigma_20'] * stdevs)
    df['sprd_dn'] = df['spread_20'] - (df['sigma_20'] * stdevs)
    df['zscore'] = (df['spread'] - df.spread_20) / df.sigma_20
    df['z_spread'] = np.where((df.zscore > stdevs),'Hi',np.where((df.zscore < -stdevs),'Lo','Avg'))
    
    ZSpread = df['z_spread'].iloc[-1]
    ZScore = df['zscore'].iloc[-1]
    
    if ('High','Low') not in df.columns:
        df = df.rename(columns={'ask':'High','bid':'Low'})
    rng = df['High'] - df['Low']
    df['ATR'] = rng.rolling(window=14).mean()
    df['z_slip'] = (df['ATR'] + df['spread'])/denom
    ZSlip = df['z_slip'].iloc[-1]
    
    return ZSlip, ZSpread,  ZScore


get_z_spread_slip(df,1,2)

NameError: name 'df' is not defined

In [67]:
def get_z_slip_multi(tickers,stdevs=1,denom=1.5):
    ZSlips = {}
    Z = {}
    for tick in tickers:
        df = get_symbol_ticks(tick)
        #df = temp_ticks(tick) -- Used YF initially in testing... not needed when IB works.
        zsl, zsp, zsc = get_z_spread_slip(df,stdevs,denom)
        ZSlips[tick] = zsl
        Z[tick] = [zsl, zsp, zsc]
    return Z

get_z_slip_multi(['AAPL','AMZN','TSLA'],2,2)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'AAPL': [6.396420070103236, 'Avg', 1.600115083585767],
 'AMZN': [40.506399972098215, 'Hi', 2.5835842612567985],
 'TSLA': [18.70536150251116, 'Avg', -1.1101054572859261]}

In [17]:
'''ZSpread in USD (based on 65% of spread)'''

def get_z_slip_multi(tickers,stdevs=1,denom=1.5):
    ZSlips = {}
    Z = {}
    for tick in tickers:
        df = get_symbol_ticks(tick)
        #df = temp_ticks(tick) -- Used YF initially in testing... not needed when IB works.
        zsl, zsp, zsc = get_z_spread_slip(df,stdevs,denom)
        ZSlips[tick] = zsl
        Z[tick] = [zsl, zsp, zsc]
    return Z

def get_realized_spread_USD(tickers,pos_size_USD,pct_realized=.65):
    USD = {}
    for i in tickers:
        df = pdr.DataReader(i,'yahoo','2020')
        n_shares = pos_size_USD / df.Close.tail(1)[0]
        print(n_shares)
        zslip , _ , _ = get_z_spread_slip(df) #Select zslip (1st unpack) or zspread (2nd unpack)
        print(zslip)
        USD[i] = (zslip*pct_realized)*n_shares
    return USD
   
import pandas_datareader as pdr
import numpy as np
get_realized_spread_USD(['AMD','TSLA'],100)    

1.8426420105705335
2.3076184590657554
0.12780369153287283
54.94713774181548


{'AMD': 2.7638745660776896, 'TSLA': 4.564590577670019}